In [14]:
import pandas as pd
import os
import numpy as np

In [15]:
# 현재 위치
current_path = os.getcwd()

# 포지션 선수 판별용 선수라벨링

## 라벨링할 대상은 2021-2022 프리미어리그 선수들, 2022-2023 프리미어리그 선수들

In [16]:
Premier_1_dir_path = os.path.join(current_path,'datas','NEW_FM_DATA','2021-2022','2021-2022-PremierLeague')
Premier_2_dir_path = os.path.join(current_path,'datas','NEW_FM_DATA','2022-2023','2022-2023-PremierLeague')

In [17]:
# 빈 데이터 프레임생성 
position_df = pd.DataFrame(columns=['이름', '포지션', '구단', '리그', '임대 기간', '승', '무', '패', '출장시간', '경기당 팀 득점/90',
       '경기당 팀 실점/90', '골', '도움', 'Cr C/A', '헤더 시도', 'Cr A', 'FA', 'Fls',
       '경기 당 드리블', '드리블', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', 'Int/90',
       '달린 거리/90분', '공중 A/90', '경기당 헤더 성공', '태클 성공률', '태클', '경기당 패스 시도/90',
       '경기당 패스 성공/90', 'K Ps/90'])


def merge_season_datas(path):
    
    global position_df
    
    # n번째 시즌 처리
    for file in os.listdir(path):

        # n번째 시즌 경로
        this_path = os.path.join(path,file)

        # 불러오기
        this_df = pd.read_csv(this_path,encoding='utf-8')

        # 포지션이 없는 쓰레기값 정리
        this_df = this_df.drop(index = this_df[this_df['포지션'].isna()].index)

        # 골키퍼는 통계 자료 부족으로 삭제
        this_df = this_df[this_df['포지션'] != 'GK']

        # 현재 구단의 이름을 추출
        team = this_df[this_df['임대 기간']=='-']['구단'].iloc[0]

        # - 데이터를 0으로 변환
        this_df = this_df.replace('-',0)

        # 현재 구단에 속해 있으며 출장 시간이 900분을 넘긴 선수만 남김
        this_df = this_df[(this_df['구단']==team) & (this_df['출장시간'].astype('int64')>900)]

        # 이어붙이기
        position_df = pd.concat([position_df,this_df],axis=0)
    
# 자료형 변경
position_df['승'].dtypes == 'int64'

merge_season_datas(Premier_1_dir_path)
merge_season_datas(Premier_2_dir_path)   

In [18]:
# 인덱스 리셋
position_df =position_df.reset_index(drop=True)

# 확인
position_df.head()

,이름,포지션,구단,리그,임대 기간,승,무,패,출장시간,경기당 팀 득점/90,...,경기당 슈팅/90,Int/90,달린 거리/90분,공중 A/90,경기당 헤더 성공,태클 성공률,태클,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90
0,Teemu Pukki,ST (C),노리치,잉글랜드 프리미어 리그,0,7,11,16,2015,0.98,...,2.32,0.76,12.3km,7.28,2.23,100%,0.58,30.64,26,0.8
1,Billy Gilmour,M (C),노리치,잉글랜드 프리미어 리그,21/7/1 - 22/5/31,8,13,16,2677,1.01,...,0.84,0.87,11.9km,2.49,1.45,77%,1.68,53.15,47.34,1.55
2,Pierre Lees-Melou,"DM, M (C)",노리치,잉글랜드 프리미어 리그,0,8,13,19,3526,0.94,...,0.74,1.35,12.4km,2.73,1.48,90%,1.33,52.53,47.14,0.82
3,Milot Rashica,"M/AM (R), ST (C)",노리치,잉글랜드 프리미어 리그,0,8,9,15,2633,1.06,...,1.03,1.57,13.0km,4.96,3.08,86%,1.5,33.29,28.4,1.57
4,Christos Tzolis,"AM (RL), ST (C)",노리치,잉글랜드 프리미어 리그,0,7,7,13,1187,0.76,...,2.5,0.76,12.1km,9.48,4.78,80%,1.21,32.6,27.52,1.52


In [54]:
# 저장
position_df.to_csv('./datas/포지션판별모델생성용데이터.csv',sep=",",index=False,header=True,encoding='utf-8')